# 02 Chat Completion | 02 .NET SDK

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.16"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
static string _configurationFile = @"../Configuration/application.env";
Env.Load(_configurationFile);


string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"OpenAI Client created...");

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.16 DotNetEnv, 2.5.0

OpenAI Client created...


## Step 1: Basic Interaction

The following cell, demonstrate a basic interaction with OpenAI Client. In this case, the system message is used to provide instructions or settings for the assistant, such as its personality or behavior. The user message is used to provide queries or inputs from the human user. The assistant message is used to provide responses or outputs from the model.

System prompt: `"You are an AI assistance who helps finding information around international sport events"`

Prompt: `"Who won the super bowl 2024?"`

In [2]:
string sys_prompt = "You are an AI assistance who helps finding information around international sport events";
ChatCompletionsOptions simpleOption = new ChatCompletionsOptions();

simpleOption.Messages.Add(new ChatRequestSystemMessage(sys_prompt));

simpleOption.Messages.Add(new ChatRequestUserMessage( "Who won the super bowl 2024?"));

//Request Properties
simpleOption.MaxTokens = 500;
simpleOption.Temperature = 0.0f;
simpleOption.NucleusSamplingFactor = 0.0f;
simpleOption.FrequencyPenalty = 0.7f;
simpleOption.PresencePenalty = 0.7f;
simpleOption.StopSequences.Add("\n"); 
simpleOption.DeploymentName = chatCompletionDeploymentName;

Response<ChatCompletions> simpleResponse = await openAIClient.GetChatCompletionsAsync(simpleOption);

// Get the first choice from the response
ChatCompletions simpleCompletions = simpleResponse.Value;
foreach (ChatChoice chatChoice in simpleCompletions.Choices) {
    Console.WriteLine($"ChatChoiceIndex: {chatChoice.Index}");
    Console.WriteLine($"Content: {chatChoice.Message.Content}");
}

ChatChoiceIndex: 0
Content: I'm sorry, but I cannot provide real-time information as my knowledge is limited to publicly available data up to September 2021. I recommend checking the latest news or sports websites for the most recent Super Bowl winner.


## Step 2: Complex interaction - Chat History

System Prompt: 
```html
    You extract intention from provided text. The two intentions you identify are: product information and order status. 
    You answer with a valid JSON string. 
    The JSON string must have the format {""Intention"": ""ProductInformation""} or {""Intention"": ""OrderInfo""}. 
    You don't provide additional information. 
    If you can't identify intention answer with {""Intention"": ""Unknown""}
```

Chat History: 

```html
    - I've purchased three weeks ago new training shoes. When will they will be delivered?
    - Still waiting for the delivery. Any idea when it will arrive? I'm Robert and I'm calling on behalf of company Contoso.
    - Do you have training shoes? If yes, I'm interested in running equipment specifically running shoes.
    - What is the average price for running shoes?
```



In [3]:
// Define System Prompt
string systemPrompt = @" 
    You extract intention from provided text. The two intentions you identify are: product information and order status. 
    You answer with a valid JSON string. 
    The JSON string must have the format {""Intention"": ""ProductInformation""} or {""Intention"": ""OrderInfo""}. 
    You don't provide additional information. 
    If you can't identify intention answer with {""Intention"": ""Unknown""}
";

// Compose Chat (Few Shot Learning)
ChatCompletionsOptions chatCompletionsOptions = new ChatCompletionsOptions();

// Add messages using the new methods
chatCompletionsOptions.Messages.Add(new ChatRequestSystemMessage(systemPrompt));

chatCompletionsOptions.Messages.Add(new ChatRequestUserMessage("I've purchased three weeks ago new training shoes. When will they will be delivered?"));
// Assuming that the assistant responses are generated by the AI and not predefined, we don't add them here

chatCompletionsOptions.Messages.Add(new ChatRequestUserMessage("Still waiting for the delivery. Any idea when it will arrive? I'm Robert and I'm calling on behalf of company Contoso."));

chatCompletionsOptions.Messages.Add(new ChatRequestUserMessage("Do you have training shoes? If yes, I'm interested in running equipment specifically running shoes."));

chatCompletionsOptions.Messages.Add(new ChatRequestUserMessage("What is the average price for running shoes?"));

// Request Properties
chatCompletionsOptions.MaxTokens = 500;
chatCompletionsOptions.Temperature = 0.0f;
chatCompletionsOptions.NucleusSamplingFactor = 0.0f;
chatCompletionsOptions.FrequencyPenalty = 0.7f;
chatCompletionsOptions.PresencePenalty = 0.7f;
chatCompletionsOptions.StopSequences.Add("\n"); 

// Deployment Name
chatCompletionsOptions.DeploymentName = chatCompletionDeploymentName;

Console.WriteLine("ChatCompletionsOptions created...");

// Get Chat Completions
Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);

// Handle the response
ChatCompletions chatCompletions = response.Value;
foreach (ChatChoice chatChoice in chatCompletions.Choices) {
    Console.WriteLine($"ChatChoiceIndex: {chatChoice.Index}");
    Console.WriteLine($"Content: {chatChoice.Message.Content}");
}


ChatCompletionsOptions created...
ChatChoiceIndex: 0
Content: {"Intention": "ProductInformation"}


## Step 3: Check additional response values

### Token Usage

Provide tokens consumed by the `GetChatCompletionsAsync()` call.

In [5]:
//Get Raw Response
var rawResponse = response.GetRawResponse();
if (!rawResponse.IsError) {
    //Get Raw Response Content
    JsonElement jsonElement = JsonSerializer.Deserialize<JsonElement>(rawResponse.Content.ToString());
    JsonElement tokenUsage = jsonElement.GetProperty("usage");
    tokenUsage.TryGetProperty("completion_tokens", out JsonElement completionTokens);
    Console.WriteLine($"Completion Tokens: {completionTokens.ToString()}");
    tokenUsage.TryGetProperty("prompt_tokens", out JsonElement promptTokens);
    Console.WriteLine($"Prompt Tokens: {promptTokens.ToString()}");
    tokenUsage.TryGetProperty("total_tokens", out JsonElement totalTokens);
    Console.WriteLine($"Total Tokens: {totalTokens.ToString()}");
}    

Completion Tokens: 2
Prompt Tokens: 198
Total Tokens: 200


### Filter 

If the LLM detects:
- hate
- self harm
- violence
- sexual
content within the provided prompt or in the result it will be filtered and reported

In [6]:
//Prompt Filter
Console.WriteLine($"Prompt Filter Results");
foreach(ContentFilterResultsForPrompt promptFilterResult in chatCompletions.PromptFilterResults) {
    Console.WriteLine($"PromptIndex: {promptFilterResult.PromptIndex}");
    Console.WriteLine($"\t ContentFilterResults: Hate");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.Hate.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.Hate.Severity}");
    Console.WriteLine($"\t ContentFilterResults: SelfHarm");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.SelfHarm.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.SelfHarm.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Violence");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.Violence.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.Violence.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Sexual");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.Sexual.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.Sexual.Severity}");
}

//Response Filter
foreach (ChatChoice chatChoice in chatCompletions.Choices) {
    Console.WriteLine($"ChatChoiceIndex: {chatChoice.Index}");
    Console.WriteLine($"\t ContentFilterResults: Hate");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.Hate.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.Hate.Severity}");
    Console.WriteLine($"\t ContentFilterResults: SelfHarm");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.SelfHarm.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.SelfHarm.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Violence");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.Violence.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.Violence.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Sexual");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.Sexual.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.Sexual.Severity}");
}

Prompt Filter Results
PromptIndex: 0
	 ContentFilterResults: Hate
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: SelfHarm
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Violence
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Sexual
		 Filtered: False
		 Severity: safe
ChatChoiceIndex: 0
	 ContentFilterResults: Hate
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: SelfHarm
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Violence
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Sexual
		 Filtered: False
		 Severity: safe
